---

## Recognizing Language C

<pre>
CSC 427, Semester 252
Burton Rosenberg
University of Miami
6 march 2025

Copyright 2025 (c) Burton Rosenberg. All rights reserved.
</pre>

---


### The Turing Machine Simulator

Please see the [Syntax](https://github.com/csc427-242/sketches/blob/main/TM_Syntax.md) description and
the [examples](https://github.com/csc427-242/sketches/blob/main/turing_machine_sketch.ipynb).

The brief introduction is that the k-tape TM is described by a text (given as a triple quoted string) that
is arranged in stanzas with a keyword introducing the type of stanza. The first few stanzas should give the 
name of the start, accept and reject states. It can also declare the number of tapes if not the default 
of a one tape machine.

Then follows the transtion table, state by state. First the state is declared followed by a possible actions,
one per line. Actions for a single tape machine are of the form,

<pre>
    read-symbol write-symbol action new-state 
</pre>

where the actions are <verb>l, r, n</verb> for move left, right or no movement. If there is no matching rule, the TM rejects.

This is a system of wildcards using the meta symbol "<verb>:</verb>". 

- When the wildcard is the read-symbol, it matches any character. Rules with wildcards have lower
priority to exact matches. 
- When the wildcard is the write-symbol, it matches the read character matched.

Other special characters to watch for:

- The underscore character "<verb>_</verb>" matches the space.
- The hash character "<verb>#</verb>" anywhere on the line introduces a comment that continues to the end of the line. The hash cannot be used as a tape symbol.

The simulator has these options,

- A maximum step count. The simulator formally rejects a string if the computation exceeds the step count. The TM object can be queried to determine if the reject was caused by exceeding the step count.
- The machine has a verbose mode. Also, capitalizing the action as <verb>L, R, N</verb> will print out the machine configuration when that rule is used.



In [76]:
from turing_machine_sim import *


tm_abc= """# a TM program to recognize a*b*c*
# mar 6, 2025

accept: A
reject: R
start: q0

state: q0
    _ : n A
    a : r q0  # match a
    b : r q1
    c : r q2
    
state: q1
    b : r q1  # match b
    c : r q2
    _ : n A

state: q2
    c : r q2  # match c
    _ : n A

"""

# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100


tm = MachineParser.create_from_description(tm_abc)
for w in ['','a','bb', 'acc', 'abc']:
    print(f'\nstring: |{w}|')
    tm.compute_tm(w, max_steps, verbose=verbose)
    
for w in ['ba','acb', 'cb', 'aabbca' ]:
    print(f'\nstring: |{w}|')
    tm.compute_tm(w, max_steps, verbose=verbose)
    



string: ||
0 [q0]	[_]
1 [A]	[_]
accept (ok)

string: |a|
0 [q0]	[a]_
1 [q0]	a[_]
2 [A]	a[_]
accept (ok)

string: |bb|
0 [q0]	[b]b_
1 [q1]	b[b]_
2 [q1]	bb[_]
3 [A]	bb[_]
accept (ok)

string: |acc|
0 [q0]	[a]cc_
1 [q0]	a[c]c_
2 [q2]	ac[c]_
3 [q2]	acc[_]
4 [A]	acc[_]
accept (ok)

string: |abc|
0 [q0]	[a]bc_
1 [q0]	a[b]c_
2 [q1]	ab[c]_
3 [q2]	abc[_]
4 [A]	abc[_]
accept (ok)

string: |ba|
0 [q0]	[b]a_
1 [q1]	b[a]_
reject (transition missing)

string: |acb|
0 [q0]	[a]cb_
1 [q0]	a[c]b_
2 [q2]	ac[b]_
reject (transition missing)

string: |cb|
0 [q0]	[c]b_
1 [q2]	c[b]_
reject (transition missing)

string: |aabbca|
0 [q0]	[a]abbca_
1 [q0]	a[a]bbca_
2 [q0]	aa[b]bca_
3 [q1]	aab[b]ca_
4 [q1]	aabb[c]a_
5 [q2]	aabbc[a]_
reject (transition missing)



### The Algorithm

The language of strings $a^ib^ic^i$ is not context free. But we can recognize it with a turning machine.

The algorithm depends on the loop invariant, that the string is of the form,

$$
\dashv \, \$x^*a^ix^*b^jx^*c^k
$$

and that any imbalance of letters is preserved in the loop.

The termination condition for a correct string is that $i=j=k=0$ and the string is,

$$
\dashv \, \$x^+
$$

(the special case of the original string being empty is handled separately), and re-enter the loop if the 
string is not of this form.

The loop update is to reduce each by $i$, $j$ and $k$ by one, and to fail if any of those indices is zero.


In [83]:
# a^i b^i c^i

tm_aibici = """# a^i b^i c^i
# mar 6, 2025 -bjr

# stage1 sets the precondition for the loop
# stage2 tests the loop condition
# stage3 maintains the loop invariant and reduces the problem by one

# built for comfort, not for speed

accept: A
reject: R
start: q0
    

# create the loop invariant

state: q0
    : : N stage1

state: stage1
    _ _ n A # accept empty string
    a $ r q1
    
state: q1
    a : r q1
    b x r q2
    
state: q2
    b : r q2
    c x r q3
    
state: q3
    c : r q3
    _ : l q4
    
state: q4
    : : l q4
    $ : N stage2


# given the loop invariant, check if done

state: stage2
    : : N r1
    
state: r1
    $ : r r1
    x : r r1
    _ : n A   # the tape is $x+ so accept
    : : l r2  # anything other than $, x or _, rewind

state: r2
    : : l r2
    $ : n stage3


# given the loop invariant, and we are not done, reduce the problem
# while maintaining the loop invariant

state: stage3
    : : n s1
    
state: s1     # find an a; for a string in the language there must be at least one
    $ : r s1
    x : r s1
    a x r s2
    
state: s2     # find a b; for a string in the language there must be at least one
    : : r s2
    b x r s3
    _ : N R

state: s3     # find a c; for a string in the language there must be at least one
    : : r s3
    c x l s4
    _ : N R

state: s4      # the loop invariant is true; rewind and go to test (stage2)
    : : l s4
    $ : n stage2

"""

In [82]:
verbose = False
max_steps = 400

tm = MachineParser.create_from_description(tm_aibici)
for w in ['','abc','aabbcc', 'aaabbbccc', 'aaaaaaabbbbbbbccccccc']:
    print(f'\nstring: |{w}|')
    res = tm.compute_tm(w, max_steps, verbose=verbose)
    print(f'accept: {res}\n\t{tm.result_reasons[tm.result]}')
    
for w in ['ba', 'cb', 'acb', 'a','ab','ac', 'aabcc', 'abbcc', 'aabbc' ]:
    print(f'\nstring: |{w}|')
    res = tm.compute_tm(w, max_steps, verbose=verbose)
    print(f'accept: {res}\n\t{tm.result_reasons[tm.result]}')


string: ||
1 [stage1]	[_]
accept: True
	ok

string: |abc|
1 [stage1]	[a]bc_
8 [stage2]	[$]xx_
9 [r1]	[$]xx_
accept: True
	ok

string: |aabbcc|
1 [stage1]	[a]abbcc_
14 [stage2]	[$]axbxc_
15 [r1]	[$]axbxc_
31 [r1]	[$]xxxxx_
accept: True
	ok

string: |aaabbbccc|
1 [stage1]	[a]aabbbccc_
20 [stage2]	[$]aaxbbxcc_
21 [r1]	[$]aaxbbxcc_
41 [r1]	[$]xaxxbxxc_
65 [r1]	[$]xxxxxxxx_
accept: True
	ok

string: |aaaaaaabbbbbbbccccccc|
1 [stage1]	[a]aaaaaabbbbbbbccccccc_
44 [stage2]	[$]aaaaaaxbbbbbbxcccccc_
45 [r1]	[$]aaaaaaxbbbbbbxcccccc_
81 [r1]	[$]xaaaaaxxbbbbbxxccccc_
121 [r1]	[$]xxaaaaxxxbbbbxxxcccc_
165 [r1]	[$]xxxaaaxxxxbbbxxxxccc_
213 [r1]	[$]xxxxaaxxxxxbbxxxxxcc_
265 [r1]	[$]xxxxxaxxxxxxbxxxxxxc_
321 [r1]	[$]xxxxxxxxxxxxxxxxxxxx_
accept: True
	ok

string: |ba|
1 [stage1]	[b]a_
accept: False
	transition missing

string: |cb|
1 [stage1]	[c]b_
accept: False
	transition missing

string: |acb|
1 [stage1]	[a]cb_
accept: False
	transition missing

string: |a|
1 [stage1]	[a]_
accept: False
	transition